In [ ]:
!pip install -q tflite-model-maker

     |████████████████████████████████| 616 kB 4.0 MB/s 
     |████████████████████████████████| 596 kB 37.5 MB/s 
     |████████████████████████████████| 840 kB 70.9 MB/s 
     |████████████████████████████████| 120 kB 67.8 MB/s 
     |████████████████████████████████| 64 kB 3.0 MB/s 
     |████████████████████████████████| 3.4 MB 11.1 MB/s 
     |████████████████████████████████| 1.1 MB 16.6 MB/s 
     |████████████████████████████████| 87 kB 6.1 MB/s 
     |████████████████████████████████| 1.2 MB 47.8 MB/s 
     |████████████████████████████████| 213 kB 57.5 MB/s 
     |████████████████████████████████| 6.3 MB 39.9 MB/s 
     |████████████████████████████████| 1.1 MB 35.7 MB/s 
     |████████████████████████████████| 25.3 MB 1.6 MB/s 
     |████████████████████████████████| 352 kB 76.0 MB/s 
     |████████████████████████████████| 99 kB 8.0 MB/s 
     |████████████████████████████████| 47.6 MB 73 kB/s 
     |████████████████████████████████| 208 kB 46.4 MB/s 


In [ ]:
import numpy as np
import os

from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.text_classifier import AverageWordVecSpec
from tflite_model_maker.text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

In [7]:
spec = model_spec.get('mobilebert_classifier')

In [8]:
data = DataLoader.from_csv(
    filename='/content/lmblog_comments1.csv',
    text_column='commenttext',
    label_column='spam',
    model_spec=spec,
    delimiter=',',
    shuffle=True,
    is_training=True)

train_data, test_data = data.split(0.8)

In [9]:
model = text_classifier.create(train_data, model_spec=spec, epochs=3, 
                               validation_data=test_data)

16/16 [==============================] - 436s 24s/step - loss: 0.6541 - test_accuracy: 0.8151 - val_loss: 0.2131 - val_test_accuracy: 0.9050
Epoch 2/2
16/16 [==============================] - 358s 22s/step - loss: 0.1882 - test_accuracy: 0.9232 - val_loss: 0.0973 - val_test_accuracy: 0.9600
Epoch 3/3
16/16 [==============================] - 366s 23s/step - loss: 0.1234 - test_accuracy: 0.9557 - val_loss: 0.0992 - val_test_accuracy: 0.9550


In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 128)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 hub_keras_layer_v1v2 (HubKeras  (None, 512)         24581888    ['input_word_ids[0][0]',         
 LayerV1V2)                                                       'input_mask[0][0]',         

In [ ]:
model.evaluate(test_data)

7/7 [==============================] - 24s 3s/step - loss: 0.0992 - test_accuracy: 0.9550


In [ ]:
model.export(export_dir='mobilebert_classifier')